In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from lmfit import minimize, Parameters
import time
import logging

The method for synthetic data generation and the core pelt algorithm is entirely adapted from :
     
      author="M. Gong, R. Killick, C. Nemeth, J. Quinton",
      title="A changepoint approach to modelling non-stationary soil moisture dynamics", 
      year="2024",
      eprint="2310.17546",
      archivePrefix="arXiv",
      url="https://arxiv.org/abs/2310.17546"



In [ ]:
time_series_length = 5000
poisson_intensity = 0.003  

def changepoints(length, intensity):
    changepoints = np.random.poisson(intensity, size=length)
    changepoints_indices = np.where(changepoints > 0)[0]
    return changepoints_indices
changepoints_scenario_1a = changepoints(time_series_length, poisson_intensity)
plt.scatter(changepoints_scenario_1a, [1] * len(changepoints_scenario_1a), c='r', label="Changepoints")
plt.title('Changepoints for Scenario 1a')
plt.xlabel('Time')
plt.ylabel('Changepoints')
plt.legend()
plt.show()

print(f"Generated Changepoints: {changepoints_scenario_1a}")


In [ ]:
drying_rate_slow = (0.99, 0.995) 
drying_rate_fast = (0.95, 0.99)   

def make_gammas(length, slow_rate_range, fast_rate_range, changepoints):
    drying_rates = np.random.uniform(*slow_rate_range, size=length)
    fast_rate_indices = np.random.choice(changepoints, size=len(changepoints) // 2, replace=False)
    drying_rates[fast_rate_indices] = np.random.uniform(*fast_rate_range, size=len(fast_rate_indices))
    return drying_rates

drying_rates_scenario_1a = make_gammas(time_series_length, drying_rate_slow, drying_rate_fast, changepoints_scenario_1a)
def drying_rates_to_gamma(drying_rates):
    return np.log(-np.log(drying_rates))
gammas_scenario_1a = drying_rates_to_gamma(drying_rates_scenario_1a)
print(f"gammas:{gammas_scenario_1a}")

In [ ]:

increment_range = (0.1, 0.12) 

def make_increments(changepoints, increment_range):
    increments = np.random.uniform(*increment_range, size=len(changepoints))
    return increments

increments_scenario_1a = make_increments(changepoints_scenario_1a, increment_range)

plt.scatter(changepoints_scenario_1a, increments_scenario_1a, color='g', label="Increments")
plt.title('Increments at Changepoints for Scenario 1a')
plt.xlabel('Time')
plt.ylabel('Increments')
plt.legend()
plt.show()
print(f"Generated Increments: {increments_scenario_1a}")

In [ ]:
asymptotic_soil_moisture_range = (0.05, 0.08)  
def make_alpha_zero(length, asymptotic_range):
    asymptotic_moisture = np.random.uniform(*asymptotic_range, size=length)
    return asymptotic_moisture
asymptotic_moisture_scenario_1a = make_alpha_zero(time_series_length, asymptotic_soil_moisture_range)
plt.plot(asymptotic_moisture_scenario_1a, label="Asymptotic Soil Moisture")
plt.title('Asymptotic Soil Moisture for Scenario 1a')
plt.xlabel('Time')
plt.ylabel('Soil Moisture Level')
plt.legend()
plt.show()

print(f"Asymptotic Soil Moisture: {asymptotic_moisture_scenario_1a}")


In [ ]:
noise_std_1 = 0.0005 
noise_std_2 = 0.001  
def generate_noise(length, std_dev):
    noise = np.random.normal(0, std_dev, size=length)
    return noise
noise_scenario_1a = generate_noise(time_series_length, noise_std_1)


plt.plot(noise_scenario_1a, label="Gaussian Noise")
plt.title('Gaussian Noise (Low) for Scenario 1a')
plt.xlabel('Time')
plt.ylabel('Noise Level')
plt.legend()
plt.show()
print(f"Generated Noise: {noise_scenario_1a[:10]} (first 10 values)")


In [ ]:
def exponential_decay_model(t, tau_i, alpha0, alpha1, gamma):
    return alpha0 + alpha1 * np.exp(-np.exp(gamma) * (t - tau_i))


def generate_time_series_scenario_1a(length, changepoints, increments, gammas, asymptotic_moisture, noise):
    time_series = np.zeros(length)
    last_changepoint = 0

    for i, changepoint in enumerate(changepoints):
        increment = increments[i]
        gamma = gammas[changepoint] 
        alpha0 = asymptotic_moisture[changepoint]
        alpha1 = increment  
    
        for t in range(last_changepoint, changepoint):
            time_series[t] = exponential_decay_model(t, last_changepoint, alpha0, alpha1, gamma)
        
        time_series[changepoint] += increment
        last_changepoint = changepoint
    
  
    for t in range(last_changepoint, length):
        time_series[t] = exponential_decay_model(t, last_changepoint, 
                                                 asymptotic_moisture[last_changepoint],
                                                 time_series[last_changepoint] - asymptotic_moisture[last_changepoint],
                                                 gammas[last_changepoint])
    

    time_series += noise
    
    return time_series

final_time_series_scenario_1a = generate_time_series_scenario_1a(
    time_series_length, changepoints_scenario_1a, increments_scenario_1a, gammas_scenario_1a,
    asymptotic_moisture_scenario_1a, noise_scenario_1a
)

plt.plot(final_time_series_scenario_1a, label="Soil Moisture Time Series")
plt.title('Corrected Soil Moisture Time Series for Scenario 1a')
plt.xlabel('Time')
plt.ylabel('Soil Moisture')
plt.legend()
plt.show()


In [ ]:
#re parameterised as per the logic in Gong et al. 2024
def exp_decay(t, alpha_0, alpha_1, gamma_i, tau_i):
    return alpha_0 + alpha_1 * np.exp(-np.exp(gamma_i) * (t - tau_i))

#calculate the residuals, 
def residual(params, t, Y, tau_i):
    alpha_0 = params['alpha_0']
    alpha_1 = params['alpha_1']
    gamma_i = params['gamma_i']
    model = exp_decay(t, alpha_0, alpha_1, gamma_i, tau_i)
    return Y - model

def retry_parameter_estimation(Y, tau_i, tau_ip1):
    initial_guesses = [
        [0.07, 0.1, -5],  
        [0.06, 0.15, -10], 
        [0.08, 0.1, -10]  
    ]
    
    params = Parameters()
    
    t = np.arange(tau_i + 1, tau_ip1 + 1)
    segment_Y = Y[tau_i:tau_ip1]
    result = minimize(residual, params, args=(t, segment_Y, tau_i), method='leastsq')
        
    if result.success:
        alpha_0_i = result.params['alpha_0'].value
        alpha_1_i = result.params['alpha_1'].value
        gamma_i = result.params['gamma_i'].value
        logging.debug(f"estimated params for seg {tau_i} to {tau_ip1}: alpha_0={alpha_0_i}, alpha_1={alpha_1_i}, gamma={gamma_i}")

        if alpha_0_i > 0 and alpha_1_i > 0:
            
            return [alpha_0_i, alpha_1_i, gamma_i]
    
    return [np.inf, np.inf, np.inf]

def nll(params, Y, tau_i, tau_ip1):
    alpha_0, alpha_1, gamma_i = params
    n = tau_ip1 - tau_i
    t = np.arange(tau_i + 1, tau_ip1 + 1)
    residuals = Y[tau_i:tau_ip1] - exponential_decay_model(t, alpha_0, alpha_1, gamma_i, tau_i)
    cost = n * np.log(2 * np.pi) + 1 + np.sum(residuals ** 2)
    return cost

def costFunction(Y, changepoints, penalty_lambda):
    k = len(changepoints) - 1
    cost = 0
    parameters = []
    
    for i in range(k):
        tau_i = changepoints[i]
        tau_ip1 = changepoints[i + 1]
        params = retry_parameter_estimation(Y, tau_i, tau_ip1)
        segment_cost = nll(params, Y, tau_i, tau_ip1)
        logging.debug(f"seg cost from {tau_i} to {tau_ip1}: {segment_cost}")

        parameters.append(params)
        cost += segment_cost

    penalty = penalty_lambda * k
    total_cost = cost + penalty
    logging.debug(f"total cost: {total_cost} (penalty for {k})")
    return total_cost, parameters

def deleteDuplicates(changepoints, min_separation=5):
    unique_changepoints = []
    changepoints = sorted(set(changepoints))
    for i in range(len(changepoints)):
        if i == 0 or (changepoints[i] - changepoints[i-1]) >= min_separation:
            unique_changepoints.append(changepoints[i])
    return unique_changepoints


def PELT(Y, min_seg_length=500, penalty_lambda=5, K=0, max_runtime=600):
    start_time = time.time()
    
    n = len(Y)
    F = np.full(n + 1, np.inf)  
    F[0] = -penalty_lambda 
    R = {0}  
    changepoints = []  
    params_list = []  
    cpt = {0: []} 

    for t in tqdm(range(2 * min_seg_length, n + 1), desc="processing", unit="points"):
   

        if time.time() - start_time > max_runtime:
            break

        min_cost = np.inf
        best_tau = None
        best_params = None  

        logging.debug(f"candidate changepoints (R): {R}")

        for tau in R:
            if t - tau >= min_seg_length:
                logging.debug(f"tau: {tau}")
                cost, params = costFunction(Y, [tau, t], penalty_lambda)

                if np.isinf(F[tau]) or np.isinf(cost):
                    logging.debug(f"skip tau {tau}")
                    continue  


                logging.debug(f"F[tau]: {F[tau]} , F[tau] + cost: {F[tau] + cost} , min_cost: {min_cost}")


                if F[tau] + cost < min_cost:
                    logging.debug(f"best_tau: {tau} , cost: {cost}")
                    min_cost = F[tau] + cost
                    best_tau = tau
                    best_params = params

        if best_tau is not None:
            F[t] = min(F[tau] + costFunction(Y, [tau+1, t], penalty_lambda)[0] for tau in R)

            changepoints.append(best_tau)
            params_list.append(best_params)  
            cpt[t] = cpt.get(best_tau, []) + [best_tau]

        St = set()

        for s in range(t, t - min_seg_length + 2, -1):
            for t_prime in range(t + 1, s):
                pruning_cost = F[t] + costFunction(Y, [t, t_prime], penalty_lambda)[0] + K
                
                if pruning_cost > F[t_prime]:
                   
                    continue
                else:
                    St.add(t_prime)

        R_t_plus_1 = {t - min_seg_length + 1}
        R_t_plus_1 |= {tau for tau in R if F[tau] + costFunction(Y, [tau, t], penalty_lambda)[0] <= F[t] + K}
        R_t_plus_1 |= St

    changepoints = deleteDuplicates(changepoints)

    return changepoints, params_list, F, cpt  

min_seg_length = 500
penalty_lambda = 5 
K = 0

detected_changepoints, estimated_params, F_values, changepoint_tracking = PELT(
    final_time_series_scenario_1a, 
    min_seg_length=min_seg_length, 
    penalty_lambda=penalty_lambda,
    K=K,
    max_runtime=2400  
)

plt.plot(final_time_series_scenario_1a, label="Soil Moisture Time Series")
for cp in detected_changepoints:
    plt.axvline(cp, color='r', linestyle='--', label=f"Changepoint at {cp}")
plt.title('Detected Changepoints for Soil Moisture')
plt.xlabel('Time')
plt.ylabel('Soil Moisture')
plt.legend()
plt.show()
